# Local LLM (Alpaca7B) with TruLens

In this example, we'll load Alpaca7B from huggingface and run inferences locally, and use langchain as our framework to hold the different parts of our application (conversation memory, the llm, prompt templates, etc.). We'll use prompt templates to prime the model to be a gardening expert and ask questions about gardening that rely on past prompts.

We will also track the quality of this model using TruLens. As we get further in the conversation, we may run into issues which we can identify and debug.

In [ ]:
'''
!pip3 install torch
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q datasets loralib sentencepiece 
!pip -q install bitsandbytes accelerate
!pip -q install langchain
!pip install xformers
!pip install trulens-eval
'''

In [2]:
from transformers import LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline
from langchain.llms import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain
import openai
import torch
from trulens_eval.tru import Tru
from trulens_eval import tru_chain
from trulens_eval.tru_feedback import Feedback
from trulens_eval.tru_feedback import OpenAI as Feedback_OpenAI
tru = Tru()

## Create Feedback Function

The first thing we should do is define the qualities of our model we care about. In this case, we primarily care if the statement returned by the LLM is relevant to the user's query. We'll use OpenAI to set up a feedback function for query-statement relevance. Make sure to add your own openai API key!

In [3]:
openai.api_key = "..."
feedback_openai = Feedback_OpenAI()
qs_relevance = Feedback(feedback_openai.qs_relevance).on(question='input', statement = 'response')

openai api: 0requests [00:00, ?requests/s]

## Loading Alpaca7B

Here we're loading a Alpaca7B using HuggingFacePipeline's from_model_id. Alpaca7B has similar performance to OpenAI's text-davinci-003, but can be run locally on your own machine.

In [18]:
from langchain import HuggingFacePipeline

local_llm = HuggingFacePipeline.from_model_id(model_id="chavinlo/alpaca-native",
    task="text-generation",
    model_kwargs={"temperature":0.6, "top_p":0.95, "max_length":256})

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

## Setting up a Chat with memory

It's also important for our AI assistant to have memory of the things we tell it. That way it can give information that is most relevant to our location, conditions, etc. and feels more like we are talking to a human.

First we'll set up our AI assistant to remember up to 4 turns in our conversation using ConversationBufferWindowMemory.

Then we'll update our prompt template to prime it as a gardening expert.

Last, we'll wrap it with truchain. You'll notice that this results in our first logs of the chain itself along with the feedback definition.

In [5]:
from langchain.chains import ConversationChain
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

# set the window memory to go back 4 turns
window_memory = ConversationBufferWindowMemory(k=4)

# create the conversation chain with the given window memory
conversation = ConversationChain(
    llm=local_llm, 
    verbose=True, 
    memory=window_memory
)

# update the conversation prompt template to prime it as a gardening expert
conversation.prompt.template = '''The following is a friendly conversation between a human and an AI gardening expert. The AI is an expert on gardening and gives recommendations specific to location and conditions. If the AI does not know the answer to a question, it truthfully says it does not know. 

Current conversation:
{history}
Human: {input}
AI:'''

# wrap with truchain to instrument it
tc_conversation = tru_chain.TruChain(conversation, chain_id='GardeningAIwithMemory_v1', tru = tru, feedbacks=[qs_relevance])

✅ chain GardeningAIwithMemory_v1 -> default.sqlite
✅ feedback def. feedback_hash_8ecafff2a89d22bff8ce000bee0ff73d -> default.sqlite


Now that we've set up our chain, we can make the first call and ask our AI gardening assistant a question!

While this takes a bit of time to run on our local machine, it's nonetheless pretty impressive that we can run such a high quality LLM locally.

In [6]:
# make the first call to our AI gardening assistant!
response, record = tc_conversation.call_with_record("I live in the pacific northwest, what can I plant in my outside garden?")
display(response)



> Entering new ConversationChain chain...


/Users/jreini/opt/anaconda3/envs/torch/lib/python3.11/site-packages/transformers/generation/utils.py:1255: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(
/Users/jreini/opt/anaconda3/envs/torch/lib/python3.11/site-packages/transformers/generation/utils.py:1349: UserWarning: Using `max_length`'s default (256) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(



> Finished chain.


{'input': 'I live in the pacific northwest, what can I plant in my outside garden?',
 'history': '',
 'response': ' You can plant a variety of things in your outdoor garden in the Pacific Northwest. Some good choices include perennials such as hostas, daylilies, and astilbes; shrubs such as rhododendrons, salal, and mahonia; and annuals such as marigolds, petunias, and impatiens. You can also grow herbs such as rosemary, thyme, and sage.'}

In [7]:
# continue the conversation!
response, record = tc_conversation.call_with_record("What kind of birds am I most likely to see?")
display(response)

✅ record record_hash_b1a69ab6163994fc3e3d4dc74665b8ce from GardeningAIwithMemory_v1 -> default.sqlite


> Entering new ConversationChain chain...
✅ feedback feedback_hash_8ecafff2a89d22bff8ce000bee0ff73d on record_hash_b1a69ab6163994fc3e3d4dc74665b8ce -> default.sqlite

> Finished chain.


{'input': 'What kind of birds am I most likely to see?',
 'history': 'Human: I live in the pacific northwest, what can I plant in my outside garden?\nAI:  You can plant a variety of things in your outdoor garden in the Pacific Northwest. Some good choices include perennials such as hostas, daylilies, and astilbes; shrubs such as rhododendrons, salal, and mahonia; and annuals such as marigolds, petunias, and impatiens. You can also grow herbs such as rosemary, thyme, and sage.',
 'response': ' You are likely to see a variety of birds in your outdoor garden, such as robins, blue jays, cardinals, hummingbirds, and sparrows.'}

✅ record record_hash_59bfc72a0d32cae370b3510ea12c41a1 from GardeningAIwithMemory_v1 -> default.sqlite


In [8]:
# keep it up!
response, record = tc_conversation.call_with_record("Thanks! Blue Jays would be awesome, what kind of bird feeder should I get to attract them?")
display(response)



> Entering new ConversationChain chain...
✅ feedback feedback_hash_8ecafff2a89d22bff8ce000bee0ff73d on record_hash_59bfc72a0d32cae370b3510ea12c41a1 -> default.sqlite


Input length of input_ids is 271, but `max_length` is set to 256. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.



> Finished chain.


{'input': 'Thanks! Blue Jays would be awesome, what kind of bird feeder should I get to attract them?',
 'history': 'Human: I live in the pacific northwest, what can I plant in my outside garden?\nAI:  You can plant a variety of things in your outdoor garden in the Pacific Northwest. Some good choices include perennials such as hostas, daylilies, and astilbes; shrubs such as rhododendrons, salal, and mahonia; and annuals such as marigolds, petunias, and impatiens. You can also grow herbs such as rosemary, thyme, and sage.\nHuman: What kind of birds am I most likely to see?\nAI:  You are likely to see a variety of birds in your outdoor garden, such as robins, blue jays, cardinals, hummingbirds, and sparrows.',
 'response': ' '}

Oh, looks like something is going wrong and our LLM stopped responding usefully. Let's run the trulens dashboard to explore what the issue might be.

In [9]:
tru.run_dashboard()

✅ record record_hash_2716543b3abc6c1b3c822011f8c5e65d from GardeningAIwithMemory_v1 -> default.sqlite
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

Exploring the dashboard, we found that quality degraded on the third call to the LLM. We've also hypothesized that there may be a conflict between our max token limit of the LLM and the 4 turn window memory.

In [21]:
from langchain import HuggingFacePipeline

local_llm = HuggingFacePipeline.from_model_id(model_id="chavinlo/alpaca-native",
    task="text-generation",
    model_kwargs={"temperature":0.6, "top_p":0.95, "max_length":400})

from langchain.memory import ConversationTokenBufferMemory

# Instead of window memory, let's use token memory to match the model token limit
token_memory = ConversationTokenBufferMemory(llm = local_llm, max_token_limit=400)

conversation = ConversationChain(
    llm=local_llm, 
    verbose=True, 
    memory=token_memory
)

# wrap with truchain to instrument your chain
tc_conversation = tru_chain.TruChain(conversation, chain_id='GardeningAIwithMemory_v2', tru = tru, feedbacks=[qs_relevance])

✅ chain GardeningAIwithMemory_v2 -> default.sqlite
✅ feedback def. feedback_hash_8ecafff2a89d22bff8ce000bee0ff73d -> default.sqlite


In [22]:
response, record = tc_conversation.call_with_record("What kind of pests I should worry about?")
display(response)



> Entering new ConversationChain chain...

> Finished chain.


{'input': 'What kind of pests I should worry about?',
 'history': '',
 'response': ' You should be aware of a variety of pests, such as ants, bees, wasps, flies, mosquitoes, and spiders. Depending on the region you live in, you may also need to protect your home from termites, rodents, or other pests.'}

✅ record record_hash_1e7b2ec56c41237b0fc22c674647b183 from GardeningAIwithMemory_v2 -> default.sqlite

In [23]:
response, record = tc_conversation.call_with_record("What kind of flowers will grow best in the northeast US?")
display(response)




> Entering new ConversationChain chain...
✅ feedback feedback_hash_8ecafff2a89d22bff8ce000bee0ff73d on record_hash_1e7b2ec56c41237b0fc22c674647b183 -> default.sqlite

> Finished chain.


{'input': 'What kind of flowers will grow best in the northeast US?',
 'history': 'Human: What kind of pests I should worry about?\nAI:  You should be aware of a variety of pests, such as ants, bees, wasps, flies, mosquitoes, and spiders. Depending on the region you live in, you may also need to protect your home from termites, rodents, or other pests.',
 'response': '  Some of the best flowers to grow in the northeast US are daisies, petunias, pansies, lavender, and hostas.'}

In [24]:
response, record = tc_conversation.call_with_record("What is the typical soil make-up in gardens in my area?")
display(response)



> Entering new ConversationChain chain...
✅ feedback feedback_hash_8ecafff2a89d22bff8ce000bee0ff73d on record_hash_dc6187f4595134490dd675fadfa6a33e -> default.sqlite

> Finished chain.


{'input': 'What is the typical soil make-up in gardens in my area?',
 'history': 'Human: What kind of pests I should worry about?\nAI:  You should be aware of a variety of pests, such as ants, bees, wasps, flies, mosquitoes, and spiders. Depending on the region you live in, you may also need to protect your home from termites, rodents, or other pests.\nHuman: What kind of flowers will grow best in the northeast US?\nAI:   Some of the best flowers to grow in the northeast US are daisies, petunias, pansies, lavender, and hostas.',
 'response': '  The typical soil make-up in gardens in your area is mostly clay with a few layers of topsoil. It is important to amend the soil with compost and other organic materials to ensure healthy plant growth.'}

✅ record record_hash_225ab49b5e731c4795b3a4ff9d406cb6 from GardeningAIwithMemory_v2 -> default.sqlite


In [25]:
response, record = tc_conversation.call_with_record("I'd like to grow a large tree in my backyard. Any recommendations that work well with the soil?")
display(response)



> Entering new ConversationChain chain...

> Finished chain.


{'input': "I'd like to grow a large tree in my backyard. Any recommendations that work well with the soil?",
 'history': 'Human: What kind of pests I should worry about?\nAI:  You should be aware of a variety of pests, such as ants, bees, wasps, flies, mosquitoes, and spiders. Depending on the region you live in, you may also need to protect your home from termites, rodents, or other pests.\nHuman: What kind of flowers will grow best in the northeast US?\nAI:   Some of the best flowers to grow in the northeast US are daisies, petunias, pansies, lavender, and hostas.\nHuman: What is the typical soil make-up in gardens in my area?\nAI:   The typical soil make-up in gardens in your area is mostly clay with a few layers of topsoil. It is important to amend the soil with compost and other organic materials to ensure healthy plant growth.',
 'response': '  Yes, some of the best trees to grow in clay soil are oaks, maples, and pines. These trees have deep roots that can help to improve the so

✅ record record_hash_d662eee14839a487e0a2f74a02150ec6 from GardeningAIwithMemory_v2 -> default.sqlite


✅ feedback feedback_hash_8ecafff2a89d22bff8ce000bee0ff73d on record_hash_d662eee14839a487e0a2f74a02150ec6 -> default.sqlite


In [27]:
response, record = tc_conversation.call_with_record("What other garden improvements should I make to complement these tree recommendations?")
display(response)



> Entering new ConversationChain chain...

> Finished chain.


{'input': 'What other garden improvements should I make to complement these tree recommendations?',
 'history': "Human: What kind of pests I should worry about?\nAI:  You should be aware of a variety of pests, such as ants, bees, wasps, flies, mosquitoes, and spiders. Depending on the region you live in, you may also need to protect your home from termites, rodents, or other pests.\nHuman: What kind of flowers will grow best in the northeast US?\nAI:   Some of the best flowers to grow in the northeast US are daisies, petunias, pansies, lavender, and hostas.\nHuman: What is the typical soil make-up in gardens in my area?\nAI:   The typical soil make-up in gardens in your area is mostly clay with a few layers of topsoil. It is important to amend the soil with compost and other organic materials to ensure healthy plant growth.\nHuman: I'd like to grow a large tree in my backyard. Any recommendations that work well with the soil?\nAI:   Yes, some of the best trees to grow in clay soil are 

✅ record record_hash_d0f0658a2694da1f07ead591ba6464a8 from GardeningAIwithMemory_v2 -> default.sqlite
✅ feedback feedback_hash_8ecafff2a89d22bff8ce000bee0ff73d on record_hash_d0f0658a2694da1f07ead591ba6464a8 -> default.sqlite


Our AI assistant now no longer runs out of tokens in memory. Wahoo!